<a href="https://colab.research.google.com/github/1kaiser/Gradient-based-depth-map-fusion/blob/main/Depth_Map_LeRes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installs a conda environment and restarts the session

In [ ]:
!python -m pip install -q condacolab
import condacolab
condacolab.install()
exit() ## for restarting the session

connecting google drive link

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_folder = '/content/drive/MyDrive/OUT/data/machine_learning_test_dataset/DepthMap'
import os
os.makedirs(drive_folder, exist_ok=True)
%cd {drive_folder}

In [ ]:
import os
 
# Read images with OpenCV.
#images= None
image_dir = str(drive_folder)+'/MEDIAPIPEinput/'
os.makedirs(image_dir, exist_ok=True)
image_dir_out = str(drive_folder)+'/annotated_images'
os.makedirs(image_dir_out, exist_ok=True)

video to franes

In [ ]:
video_location = '/content/drive/MyDrive/OUT/data/machine_learning_test_dataset/test.mp4'


In [ ]:

frame_rate = 25
!ffmpeg -y -hwaccel cuvid \
  -i {video_location} \
  -r {frame_rate} {image_dir}'out_%09d.png'

clones the github repository >>> creates the conda environment >>> updates the environment with necessary libraryriies

In [ ]:
%cd {drive_folder}
!git clone https://github.com/1kaiser/Gradient-based-depth-map-fusion.git
%cd Gradient-based-depth-map-fusion
!conda create --name GBDF
!conda activate GBDF
!conda env update -f GBDF.yaml
!python -m pip install opencv-python==4.6.0.66 torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install tqdm tensorboard setuptools==59.5.0 matplotlib scipy pip install timm==0.4.12 mmcv==0.6.2


Download Fusion Model checkpoints <<<

In [ ]:
model_dir = str(drive_folder)+'/Gradient-based-depth-map-fusion/models'
import os
os.makedirs(model_dir, exist_ok=True)
!wget https://github.com/1kaiser/Gradient-based-depth-map-fusion/releases/download/v0.1/model_dict.pt -O {model_dir}'/model_dict.pt'

In [ ]:
#@title Resnet Model >>> segmentation model selection Box { vertical-output: true, display-mode: "form" }
model_selection = "res101.pth" #@param ["res101.pth", "res50.pth"] {allow-input: true}
yolov5_model_path = 'LeRes/'+str(model_selection)
!wget https://github.com/1kaiser/Gradient-based-depth-map-fusion/releases/download/v0.1/{model_selection} -O {yolov5_model_path}


**running the python Script 📜 on input files in a folder**

In [ ]:
%cd {drive_folder}
%cd Gradient-based-depth-map-fusion
out_dir = image_dir_out
!mkdir -p {out_dir}

!python run.py \
-p LeRes101 \
-i {image_dir} \
-o {out_dir} \
-m 'models/model_dict.pt'

In [ ]:
out_000000001.png
out_000000001.png

In [ ]:
import os
import glob

img_names = glob.glob(os.path.join(image_dir, "*"))
img_names.sort()
img_names

out_img_names = glob.glob(os.path.join(out_dir, "*"))
out_img_names.sort()
out_img_names[2][-13:-4]
out_img_names[2][-13:-4] in img_names
g = []
for i in range(len(img_names)):
  g.append(img_names[i][-13:-4])
g
out_img_names[2][-13:-4] not in g


**🖼️image frames to video🎥**

In [ ]:
%cd /content/

In [ ]:
frame_rate = 25
#create video 1
!ffmpeg \
  -framerate {frame_rate} \
  -pattern_type glob \
  -i {image_dir}'/*.png' \
  inputA.mp4
#create vodeo 2
!ffmpeg \
  -framerate {frame_rate} \
  -pattern_type glob \
  -i {image_dir_out}'/*.png' \
  inputB.mp4
#merge video 1 and video 2
!ffmpeg \
  -i inputA.mp4 -i inputB.mp4 \
  -filter_complex \
  "[0:v][1:v]vstack=inputs=2" \
  finalOutput.mp4
#add sound of main video to final output video
!ffmpeg \
  -i 'finalOutput.mp4' -i {video_location} \
  -c:v copy -c:a copy -map 0:v:0 -map 1:a:0 \
  OUTPUT_FILE.mp4

In [ ]:
# !ffmpeg -ss 00:00:00 -t 5 -i '/content/OUTPUT_FILE.mp4' -filter_complex "[0:v] palettegen" palette.png
# !ffmpeg -ss 00:00:00 -t 5 -i '/content/OUTPUT_FILE.mp4' -i palette.png -filter_complex "[0:v] fps=10,scale=720:-1 [new];[new][1:v] paletteuse" output.gif